## Проект

Импортируем необходимые для построения и тестирования модели библиотеки.

In [694]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import dill

from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import roc_auc_score

В качестве датасета для построения модели будем использовать данные по оттоку клиентов телефонного оператора Orange: https://www.kaggle.com/mnassrib/telecom-churn-datasets

Датасет разбит на две части, будем обучаться на большей выборке, а затем 20% данных используем для тестирования построенной модели. В качестве метрики для оценки качества модели будем использовать площадь под ROC-кривой.

In [695]:
df_train = pd.read_csv('./model/churn-bigml-80.csv')
df_train.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


Видим, что в датасете присутствуют как количественные, так и категориальные переменные. 

Прежде всего преобразуем целевую переменную к числовому формату. 

In [696]:
df_train['Churn'] = np.where(df_train['Churn'] == False, 0, 1)
df_train['Churn'].value_counts()

0    2278
1     388
Name: Churn, dtype: int64

Посмотрим на тестовый датасет и проведем аналогичное преобразование целевой переменной.

In [697]:
df_test = pd.read_csv('./model/churn-bigml-20.csv')
df_test.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,False
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,True
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,True
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,False
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,False


In [698]:
df_test['Churn'] = np.where(df_test['Churn'] == False, 0, 1)
df_test['Churn'].value_counts()

0    572
1     95
Name: Churn, dtype: int64

Создадим класс `FeatureGenerator`, который будет отвечать за создание новых признаков на основе существующих.

In [699]:
class FeatureGenerator(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_mod = X.copy()
        X_mod.loc[:, 'Total charge'] = (X.loc[:, 'Total day charge'] + 
                                        X.loc[:, 'Total eve charge'] + 
                                        X.loc[:, 'Total night charge'] + 
                                        X.loc[:, 'Total intl charge'])
        X_mod.loc[:, 'Total charge**2'] = X_mod.loc[:, 'Total charge'] ** 2
        X_mod.loc[:, 'Total x CS Calls'] = X.loc[:, 'Total day calls'] * X.loc[:, 'Customer service calls']
        return X_mod

Создадим также классы, которые будут в дальнейшем использоваться при построении пайплайна для обучения модели.

In [700]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class BinaryEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return pd.DataFrame(np.where(X == 'No', 0, 1))

Отберем признаки, на которых будем обучать нашу модель.

In [701]:
continuous_columns = ['Account length', 'Number vmail messages', 'Total day minutes',  'Total day calls', 
                      'Total eve minutes', 'Total eve calls', 'Total night minutes',  'Total night calls', 
                      'Total intl minutes', 'Total intl calls', 'Customer service calls', 'Total charge', 
                      'Total charge**2',  'Total x CS Calls']

categorical_columns = ['International plan']

Создадим пайплайн для модели, гиперпараметры были отобраны отдельно с помощью перебора по сетке.

In [702]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('encoder', BinaryEncoder())
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col))
            ])
    final_transformers.append((cont_col, cont_transformer))
    

generator = FeatureGenerator()
feats = FeatureUnion(final_transformers)

In [703]:
pipeline = Pipeline([
    ('generator', generator),
    ('features', feats),
    ('classifier', GradientBoostingClassifier(random_state = 42, 
                                              max_depth=2, 
                                              n_estimators=70, 
                                              learning_rate=0.09)),
])

Создадим обучающий и тестовый датасеты на основе имеющихся данных.

In [704]:
y_train = df_train['Churn']
X_train = df_train.drop(columns=['Churn'])
y_test = df_test['Churn']
X_test = df_test.drop(columns=['Churn'])

Обучаем пайплайн.

In [705]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('generator', FeatureGenerator()),
                ('features',
                 FeatureUnion(transformer_list=[('International plan',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='International '
                                                                                         'plan')),
                                                                 ('encoder',
                                                                  BinaryEncoder())])),
                                                ('Account length',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Account '
                                                                                     'length'))])),
                                                ('Number vmai

Строим предсказания для тестовой выборки и оцениваем метрику качества модели.

In [706]:
preds = pipeline.predict_proba(X_test)[:, 1]
print(f'ROC-AUC score: {roc_auc_score(y_test, preds):.6f}')

ROC-AUC score: 0.942308


Сохраним построенную модель.

In [707]:
with open('./model/gb_model.dill', 'wb') as f:
    dill.dump(pipeline, f)

На этом подготовительная часть завершена - модель создана. Теперь необходимо запаковать ее и настроить доступ к ней с помощью API.

Выберем какое-нибудь из наблюдений тестового датасета и рассчитаем для него вероятность оттока. Это пригодится, чтобы проверить корректность работы API.

In [708]:
X_test.loc[20, :]

State                        CO
Account length              141
Area code                   415
International plan           No
Voice mail plan             Yes
Number vmail messages        32
Total day minutes         148.6
Total day calls              91
Total day charge          25.26
Total eve minutes         131.1
Total eve calls              97
Total eve charge          11.14
Total night minutes       219.4
Total night calls           142
Total night charge         9.87
Total intl minutes         10.1
Total intl calls              1
Total intl charge          2.73
Customer service calls        1
Name: 20, dtype: object

In [709]:
preds = pipeline.predict_proba(X_test)
preds[20, 1]

0.052617927131014326